In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Load datasets
ratings = pd.read_csv('/content/updated_ratings.csv')  # userId, movieId, rating
movies = pd.read_csv('/content/unique_ids_file.csv')   # movieId, title, genres, cast, crew
sentiments = pd.read_csv('/content/nique_ids_file.csv')  # movieId, Sentiment_score, title

In [ ]:
# Step 2: Merge sentiment data into movies DataFrame
movies = movies.merge(sentiments[['movieId', 'Sentiment_score']], on='movieId', how='left')

# Step 3: Preprocess metadata (split genres, cast, and crew into lists)
def preprocess_metadata(movies):
    movies['genres'] = movies['genres'].str.split('|')  # Genres are pipe-separated
    movies['cast'] = movies['cast'].str.split('|')      # Cast is pipe-separated
    movies['crew'] = movies['crew'].str.split('|')      # Crew is pipe-separated
    return movies

movies = preprocess_metadata(movies)

In [ ]:
# Step 4: Encode features for content-based filtering
def encode_features(movies):
    movies['combined_features'] = (
        movies['genres'].astype(str) + " " +
        movies['cast'].astype(str) + " " +
        movies['crew'].astype(str)
    )
    return movies

movies = encode_features(movies)

In [ ]:
# Step 5: Compute content similarity using Bag of Words (BoW)
def compute_content_similarity_bow(movies):
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(movies['combined_features'])
    similarity_matrix = cosine_similarity(bow_matrix, bow_matrix)
    return pd.DataFrame(similarity_matrix, index=movies['movieId'], columns=movies['movieId'])

content_similarity = compute_content_similarity_bow(movies)

In [ ]:
# Step 6: Compute collaborative similarity (Item-to-Item Cosine Similarity)
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)
collaborative_similarity = cosine_similarity(user_movie_matrix.T)  # Transpose to compare movies
collaborative_similarity_df = pd.DataFrame(collaborative_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

In [ ]:
# Step 7: Compute sentiment similarity based on sentiment scores
def compute_sentiment_similarity(movies):
    sentiment_scores = movies.set_index('movieId')['Sentiment_score']
    max_score = sentiment_scores.max()
    sentiment_matrix = max_score - np.abs(sentiment_scores.values[:, None] - sentiment_scores.values)
    return pd.DataFrame(sentiment_matrix, index=sentiment_scores.index, columns=sentiment_scores.index)

sentiment_similarity = compute_sentiment_similarity(movies)

In [ ]:
# Step 8: Prepare training data for neural network
def prepare_nn_data(ratings, content_similarity, collaborative_similarity, sentiment_similarity):
    data = []
    for _, row in ratings.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        actual_rating = row['rating']

        # Check if movie_id exists in all similarity matrices
        if movie_id in content_similarity.index and movie_id in collaborative_similarity.index and movie_id in sentiment_similarity.index:
            content_score = content_similarity.loc[movie_id].mean()  # Average similarity score for the movie
            collaborative_score = collaborative_similarity.loc[movie_id].mean()
            sentiment_score = sentiment_similarity.loc[movie_id].mean()
            data.append([content_score, collaborative_score, sentiment_score, actual_rating])

    return pd.DataFrame(data, columns=['content_score', 'collaborative_score', 'sentiment_score', 'actual_rating'])

In [ ]:
# Prepare the data
nn_data = prepare_nn_data(ratings, content_similarity, collaborative_similarity_df, sentiment_similarity)
X = nn_data[['content_score', 'collaborative_score', 'sentiment_score']]
y = nn_data['actual_rating']

# Normalize features for better training
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Step 9: Build the neural network
def build_nn():
    model = Sequential([
        Input(shape=(3,)),  # Three inputs: content_score, collaborative_score, sentiment_score
        Dense(16, activation='relu'),  # Hidden layer with 16 neurons
        Dense(8, activation='relu'),   # Another hidden layer with 8 neurons
        Dense(1, activation='linear')  # Output layer for predicting ratings
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])
    return model

# Create the model
nn_model = build_nn()

In [ ]:
# Step 10: Train the neural network
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step - loss: 2.2287 - mse: 2.2287 - val_loss: 1.0568 - val_mse: 1.0568
Epoch 2/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 1.0567 - mse: 1.0567 - val_loss: 1.0540 - val_mse: 1.0540
Epoch 3/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 1.0582 - mse: 1.0582 - val_loss: 1.0497 - val_mse: 1.0497
Epoch 4/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 1.0541 - mse: 1.0541 - val_loss: 1.0504 - val_mse: 1.0504
Epoch 5/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 1.0587 - mse: 1.0587 - val_loss: 1.0538 - val_mse: 1.0538
Epoch 6/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - loss: 1.0479 - mse: 1.0479 - val_loss: 1.0502 - val_mse: 1.0502
Epoch 7/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 1.0503 - mse: 1.0503 - val_loss: 1.0485 - val_mse: 1.0485
Epoch 8/20
10571/10571 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 1.0507 - mse: 1.0507 - val_loss: 1.0501 - val_mse: 1.0501
Epoch 9/20
10571

In [ ]:
# Recommendation function based on user input
def recommend_movies_by_movie(input_movie, movies, content_similarity, collaborative_similarity, sentiment_similarity, nn_model, scaler, top_n=5):
    if isinstance(input_movie, str):  # If input is a title
        input_movie_id = movies[movies['title'].str.lower() == input_movie.lower()]['movieId'].values
        if len(input_movie_id) == 0:
            print(f"Movie '{input_movie}' not found in the dataset. Please try another title.")
            return None
        input_movie_id = input_movie_id[0]
    else:  # If input is an ID
        input_movie_id = input_movie

    # Check if the input movie exists in all similarity matrices
    if (input_movie_id not in content_similarity.index or
        input_movie_id not in collaborative_similarity.index or
        input_movie_id not in sentiment_similarity.index):
        print(f"Movie ID {input_movie_id} not found in the similarity matrices. Please try another movie.")
        return None

    predicted_scores = []
    for movie_id in content_similarity.index:
        # Ensure movie_id exists in all similarity matrices
        if (movie_id != input_movie_id and
            movie_id in collaborative_similarity.index and
            movie_id in sentiment_similarity.index):
            content_score = content_similarity.loc[input_movie_id, movie_id]
            collaborative_score = collaborative_similarity.loc[input_movie_id, movie_id]
            sentiment_score = sentiment_similarity.loc[input_movie_id, movie_id]

            # Prepare the input for the neural network
            input_features = scaler.transform([[content_score, collaborative_score, sentiment_score]])
            predicted_score = nn_model.predict(input_features)[0][0]
            predicted_scores.append((movie_id, predicted_score))

    predicted_scores = sorted(predicted_scores, key=lambda x: x[1], reverse=True)

    # Get the top N recommended movies
    recommended_movie_ids = [movie_id for movie_id, _ in predicted_scores[:top_n]]
    recommendations = movies[movies['movieId'].isin(recommended_movie_ids)][['movieId', 'title']]

    # Add predicted scores to the recommendations
    recommendations['predicted_score'] = recommendations['movieId'].map(dict(predicted_scores))

    return recommendations.sort_values(by='predicted_score', ascending=False)


# Get user input and provide recommendations
def get_user_input_and_recommendations():
    input_movie = input("Enter a movie title or movieId to get recommendations: ").strip()

    try:
        # Check if the input is a valid integer (movieId)
        input_movie_id = int(input_movie)
        recommendations = recommend_movies_by_movie(
            input_movie=input_movie_id,
            movies=movies,
            content_similarity=content_similarity,
            collaborative_similarity=collaborative_similarity_df,
            sentiment_similarity=sentiment_similarity,
            nn_model=nn_model,
            scaler=scaler,
            top_n=5
        )
    except ValueError:
        # If not a valid integer, treat as a title
        recommendations = recommend_movies_by_movie(
            input_movie=input_movie,
            movies=movies,
            content_similarity=content_similarity,
            collaborative_similarity=collaborative_similarity_df,
            sentiment_similarity=sentiment_similarity,
            nn_model=nn_model,
            scaler=scaler,
            top_n=5
        )

    if recommendations is not None:
        print(f"Top 5 movies similar to '{input_movie}':")
        print(recommendations)

# Call the function to get user input and recommend movies
get_user_input_and_recommendations()

Enter a movie title or movieId to get recommendations: avatar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Top 5 movies similar to 'avatar':
      movieId               title  predicted_score
63      79132           Inception         7.896632
1169    76251  30 Minutes or Less         7.761469
105     68358           Star Trek         7.737317
43      68954                  Up         7.628722
1054    70286        Mystic River         7.592175


In [ ]:
import math  # Import math for RMSE calculation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Evaluate model performance using RMSE
y_pred = nn_model.predict(X_scaled)
rmse = math.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE of the hybrid recommender system: {rmse:.2f}")

13213/13213 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step
RMSE of the hybrid recommender system: 1.02


In [ ]:
# Function to calculate Top-N accuracy
def calculate_top_n_accuracy(predicted_ratings, ratings, top_n=5, threshold=1.0):
    """
    Calculate the Top-N accuracy for the recommender system based on predicted ratings.

    Parameters:
    - predicted_ratings: List of tuples (movie_id, predicted_rating)
    - ratings: DataFrame containing the actual ratings (userId, movieId, rating)
    - top_n: Number of recommendations to consider
    - threshold: Maximum allowable error for a correct prediction (e.g., within 1 point)

    Returns:
    - accuracy: Proportion of correct predictions within the top N
    """
    correct_predictions = 0
    total_predictions = 0

    # For each user in the ratings data
    for user_id in ratings['userId'].unique():
        user_ratings = ratings[ratings['userId'] == user_id]

        # Get the top N predicted movies for the user
        user_predicted_ratings = [(movie_id, rating) for movie_id, rating in predicted_ratings if movie_id in user_ratings['movieId'].values]
        user_predicted_ratings.sort(key=lambda x: x[1], reverse=True)  # Sort by predicted rating

        top_n_predicted = user_predicted_ratings[:top_n]

        # Compare the top N predicted movies with the actual ratings
        for movie_id, predicted_rating in top_n_predicted:
            actual_rating = user_ratings[user_ratings['movieId'] == movie_id]['rating'].values
            if len(actual_rating) > 0:
                actual_rating = actual_rating[0]
                if abs(predicted_rating - actual_rating) <= threshold:  # Check if predicted rating is within threshold
                    correct_predictions += 1
            total_predictions += 1

    # Calculate accuracy as the proportion of correct predictions
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

# Predict ratings using the trained model
predicted_ratings = predict_ratings(content_similarity, collaborative_similarity_df, sentiment_similarity, nn_model, scaler)

# Evaluate Top-N accuracy
top_n_accuracy = calculate_top_n_accuracy(predicted_ratings, ratings, top_n=5, threshold=1.0)
print(f"Top-5 Accuracy of the hybrid recommender system: {top_n_accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Top-5 Accuracy of the hybrid recommender system: 63.58%
